In [14]:
import pymongo
client=pymongo.MongoClient('192.168.1.113',27017,username="j122085",password="850605")
db=client.rawData
print(db.collection_names())
collection=db.wowprimediendata
wowDatas=list(collection.find({"avgDailyCustomer":{"$gte":0},"avgDailyNet":{"$gte":0},"costPower_Analyze":{"$gte":0}}))

['busData', 'HRdata104', 'wowprimediendata', 'departmentStore', 'smallStyleCount', 'addressCoordinate', 'Nhuman', 'ipeenWebsite', 'bigStyleCount', 'ipeenInfo', 'conStore', 'CostPower']


In [15]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
df=pd.DataFrame(wowDatas)
df.salary = df.avgDailyCustomer.astype(float)                   #traform into float type
df.working = df.avgDailyNet.astype(float)                 #traform into float type
X = df[['costPower_Analyze','Nhuman_Analyze','NbusStation_Analyze','NconStore_Analyze','Nstar_Analyze', 'Nmc_Analyze', 'Nken_Analyze','Nwa_Analyze']].values                   #tranform DataFrame to ndarray Matrix  為了predict輸入的方式
xx=X
#將每個欄位的數值都變成0-1(除以最大的數做正規化、並只留下該數值List) 
x=[]
for i in range(len(X.T)):
    x.append(X.T[i]/max(X.T[i]))
    
x=numpy.array(x)
X=x.T




In [ ]:
len(X)

In [ ]:
from sklearn import cluster, datasets, metrics
import matplotlib.pyplot as plt

# 印出效用最高的kmeans群
silhouette_avgs = []
ks = range(2, 30)
for k in ks:
    kmeans_fit = cluster.KMeans(n_clusters = k).fit(X)
    cluster_labels = kmeans_fit.labels_
    silhouette_avg = metrics.silhouette_score(X, cluster_labels) #組間變異
    silhouette_avgs.append(silhouette_avg)

# 作圖並印出 k = 2 到 10 的績效
plt.bar(ks, silhouette_avgs)
plt.show()
print(silhouette_avgs)

In [ ]:
km = KMeans(n_clusters=2)  #K=2群
y_pred = km.fit_predict(X)
# plt.figure(figsize=(10, 6))
# plt.xlabel('ADS')
# plt.ylabel('ADGC')
# plt.scatter(X[:, 0], X[:, 1], c=y_pred) #C是第三維度 已顏色做維度
# plt.show()
# km.cluster_centers_ #各群中心點(X,Y)的位置

#km.labels_是分群結果list、加入資料中
TwowDatas=[i for i in wowDatas if 'avgDailyCustomer' in i and "avgDailyNet" in i and 'costPower_Analyze' in i]
for i,j in zip(km.labels_,TwowDatas):
    j['cluster']=i

In [ ]:
#匯出成excel
dfNew=pd.DataFrame(TwowDatas)
wr=pd.ExcelWriter("ClusterWow.xlsx")
dfNew.to_excel(wr)
wr.save()


In [ ]:
#看看每群的feature分布
xxx=['avgDailyNet','avgDailyCustomer','costPower_Analyze','Nhuman_Analyze','NbusStation_Analyze','NconStore_Analyze','Nstar_Analyze', 'Nmc_Analyze', 'Nken_Analyze','Nwa_Analyze']
clusterData={}
for n in range(len(set(km.labels_))):
    clusterData[n]={}
    for i in [d for d in TwowDatas if d['cluster']==n]:
        for j in xxx:
            if i[j] >clusterData[n].get(j+"_max",0):
                clusterData[n][j+"_max"]=i[j]
            if i[j] <clusterData[n].get(j+"_min",5000000000):
                clusterData[n][j+"_min"]=i[j]
import pprint
pprint.pprint(clusterData)

In [16]:
Y=df['avgDailyCustomer'].values
len(Y)

260

In [ ]:
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers.core import Dense,Activation
from keras.utils import np_utils
import pymongo

In [ ]:
input_size=8
batch_size=5
epochs=500

model=Sequential()  #定義model
# model.add(Dense(20,input_dim=input_size)) 
model.add(Dense(50,input_dim=input_size,activation="relu")) 
model.add(Dense(30,input_dim=input_size,activation="relu")) 
model.add(Dense(10,input_dim=input_size,activation="linear")) 
model.add(Dense(1))
# model.add(Activation('relu')) #啟動函數
# model.add(Dense(50)) 
# model.add(Activation('relu')) #啟動函數
# model.add(Dense(40)) 
# model.add(Activation('sigmoid')) #啟動函數
# model.add(Dense(20)) 
# model.add(Activation('sigmoid')) #啟動函數
# model.add(Dense(1))  #加入層(緊密層) 產出個數10.輸入個數100 次元
# model.add(Activation("linear"))
# model.compile(loss="MSE",metrics=['accuracy'],optimizer='sgd')

model.compile(optimizer='rmsprop', loss='mse', metrics=['mae'])
model.fit(xx,Y,batch_size=batch_size,nb_epoch=epochs,verbose=1)


score=model.evaluate(xx,Y,verbose=1)
print('Test accuracy:',score[1])


newData=[[int(j)for j in i.split("\t")] for i in """63	282141	204	187	16	14	2	1""".split('\n')]
model.predict(numpy.array(newData).astype('float32'))

In [ ]:
model.predict(xx)

In [7]:
import numpy as np
newData=[[int(j)for j in i.split("\t")] for i in """76	370742	166	303	31	15	6	2
65	263596	130	131	13	7	0	1
46	52077	26	14	0	1	1	0
75	227620	172	144	12	12	1	2
51	173145	137	97	6	7	2	3
63	47594	4	25	0	1	0	0
63	76863	67	33	0	1	2	0""".split('\n')]
to_be_predicted = np.array(newData)
predicted_sales = model.predict(to_be_predicted)
predicted_sales

array([[422.8833 ],
       [370.45847],
       [184.84967],
       [388.81622],
       [316.43097],
       [185.45628],
       [245.41924]], dtype=float32)

In [ ]:
#存取model
from keras.models import model_from_json

#將model格式轉換成json字串，並且將其存成文字檔
json_string = model.to_json()
with open("modelJsonString",'w') as f:
    f.write(json_string)
    
#將model的訓練weight存程某檔案('modelWeight')
model.save_weights('modelWeight')

In [3]:
#讀取過去存放的資料
from keras.models import Sequential
model=Sequential()
from keras.models import model_from_json
with open("modelJsonString",'r') as f:
    json_string=f.read()

model = model_from_json(json_string)
model.load_weights('modelWeight', by_name=False)

In [ ]:
#http://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html
data=[[i['NbusStation_Analyze'],
  i['NconStore_Analyze'],
  i['costPower_Analyze'],
  i['Nhuman_Analyze']]for i in wowDatas if 'NbusStation_Analyze' in i and 'avgDailyCustomer' in i and 'costPower_Analyze' in i]

In [ ]:
data=[[i['Nstar_Analyze'],
  i['NconStore_Analyze'],
  i['Nmc_Analyze'],
  i['Nken_Analyze']]for i in wowDatas if 'NbusStation_Analyze' in i and 'avgDailyCustomer' in i]

In [ ]:
data=[[i['NbusStation_Analyze'],
  i['NconStore_Analyze'],
  i['costPower_Analyze'],
  i['Nhuman_Analyze'],
  i['Nstar_Analyze'],
  i['Nmc_Analyze'],
  i['Nken_Analyze']]for i in wowDatas if 'NbusStation_Analyze' in i and 'avgDailyCustomer' in i and 'costPower_Analyze' in i]

In [ ]:
mC=max([i['avgDailyCustomer'] for i in wowDatas if 'NbusStation_Analyze' in i and 'avgDailyCustomer' in i and 'costPower_Analyze' in i])
mN=max([i['avgDailyNet'] for i in wowDatas if 'NbusStation_Analyze' in i and 'avgDailyCustomer' in i and 'costPower_Analyze' in i])

In [ ]:
data=[[i['avgDailyNet']/mN,
  i['avgDailyCustomer']/mC] for i in wowDatas if 'NbusStation_Analyze' in i and 'avgDailyCustomer' in i and 'costPower_Analyze' in i]

In [ ]:
X = np.array(data)
from sklearn import cluster, datasets, metrics
import matplotlib.pyplot as plt

# 迴圈
silhouette_avgs = []
ks = range(2, 40)
for k in ks:
    kmeans_fit = cluster.KMeans(n_clusters = k).fit(X)
    cluster_labels = kmeans_fit.labels_
    silhouette_avg = metrics.silhouette_score(X, cluster_labels)
    silhouette_avgs.append(silhouette_avg)

# 作圖並印出 k = 2 到 10 的績效
plt.bar(ks, silhouette_avgs)
plt.show()
print(silhouette_avgs)

In [ ]:
from sklearn.cluster import KMeans
import numpy as np
X = np.array(data)
kmeans = KMeans(n_clusters=7, random_state=0).fit(X)
kmeans.labels_

In [ ]:
kmeans.cluster_centers_

In [ ]:
mainData=[i for i in wowDatas if 'NbusStation_Analyze' in i and 'avgDailyCustomer' in i and 'costPower_Analyze' in i]

In [ ]:
for i,j in zip(mainData,list(kmeans.labels_)):
    i['k']=j

In [ ]:
mainData[0]

In [ ]:
for k in range(0,51):
    print(k,[i['Called'] for i in mainData if i['k']==k])
    print("="*50)
    #'avgDailyCustomer' 'Called' 'avgDailyNet'

In [ ]:
[[i['NbusStation_Analyze'],
  i['NconStore_Analyze'],
  i['costPower_Analyze'],
  i['Nhuman_Analyze'],
  i['Nstar_Analyze'],
  i['Nmc_Analyze'],
  i['Nken_Analyze'],
  i['Called'],
  i['avgDailyNet'],
  i['avgDailyCustomer']
  ] for i in mainData if i['k']==2]

# try linear(回歸)

In [ ]:
import pymongo
client=pymongo.MongoClient('192.168.1.113',27017,username="j122085",password="850605")

db=client.rawData
print(db.collection_names())
collection=db.wowprimediendata
wowDatas=list(collection.find({}))


store=input("輸入店家")
import numpy as np
from sklearn.linear_model import LinearRegression
X= np.array([[i['costPower_Analyze'],
              i['Nhuman_Analyze'],
              i['NbusStation_Analyze'],
              i['NconStore_Analyze'],
              i['Nstar_Analyze'],
              i['Nmc_Analyze'],
              i['Nken_Analyze'],
              i['Nwa_Analyze']]for i in wowDatas if 'NbusStation_Analyze' in i and 'avgDailyCustomer' in i and 'costPower_Analyze' in i
            and i['Called']==store])

#,  i['Nstar_Analyze'],  i['Nmc_Analyze'],  i['Nken_Analyze'],  i['costPower_Analyze'],    i['Nhuman_Analyze'],    i['NbusStation_Analyze']  i['NconStore_Analyze']

Y= np.array([i['avgDailyCustomer'] for i in wowDatas if 'NbusStation_Analyze' in i and 'avgDailyCustomer' in i and 'costPower_Analyze' in i
            and i['Called']==store])

#'avgDailyCustomer' 'Called' 'avgDailyNet'

lm = LinearRegression()
lm.fit(X, Y)

# 印出係數
print(lm.coef_)
print('消費力	人口數	公車站數	四大超商數	星巴克數	麥當勞數	肯德基數	瓦城數')
# # 印出截距
# print(lm.intercept_ )


In [ ]:
# x=[int(i) for i in "58	176822	182	130	3	5	3	1".split("\t")]
# 新店資料
newData=[[int(j)for j in i.split("\t")] for i in """25\t17957\t75\t49\t0\t0\t0\t0""".split('\n')]


to_be_predicted = np.array(newData)

predicted_sales = lm.predict(to_be_predicted)

# 預測新店的客量
print(predicted_sales)

In [ ]:
# {'CooK BEEF!',
#  'hot 7',
#  'ita義塔',
#  '乍牛',
#  '原燒',
#  '品田牧場',
#  '夏慕尼',
#  '沐越',
#  '王品',
#  '石二鍋',
#  '聚',
#  '舒果',
#  '莆田',
#  '藝奇',
#  '陶板屋',
#  '青花驕',
#  '麻佬大',
#  'ＴＡＳＴｙ'}

# try neural_network (sklearn)

In [ ]:
from sklearn.neural_network.multilayer_perceptron import MLPClassifier
# from sklearn.neural_network.multilayer_perceptron import MLPRegressor

In [ ]:
import pymongo
client=pymongo.MongoClient('192.168.1.113',27017,username="j122085",password="850605")

db=client.rawData
print(db.collection_names())
collection=db.wowprimediendata
wowDatas=list(collection.find({}))

Calls=set(i["Called"] for i in wowDatas)
for Call in Calls:
    mean=np.mean([i["avgDailyCustomer"] for i in wowDatas if i["Called"]==Call and 'avgDailyCustomer' in i])
    if not np.isnan(mean):
        for j in wowDatas:
            if j["Called"]==Call and 'avgDailyCustomer' in j:
                if j["avgDailyCustomer"]>mean*1.1:
                    j['type']=0#"good"
                elif j["avgDailyCustomer"]<mean*0.9:
                    j['type']=2#"bad"
                else:
                    j['type']=1#"normal"
            if 'avgDailyCustomer' in j:
                aC=j["avgDailyNet"]/j["avgDailyCustomer"]
                if aC<290:
                    j['typeP']="低"
                elif aC<450:
                    j['typeP']="中"
                elif aC<800:
                    j['typeP']="中高"
                else:
                    j['typeP']="高"
                

store=input("輸入店家")
import numpy as np
from sklearn.linear_model import LinearRegression
X= np.array([[i['costPower_Analyze'],
              i['Nhuman_Analyze'],
              i['NbusStation_Analyze'],
              i['NconStore_Analyze'],
              i['Nstar_Analyze'], 
              i['Nmc_Analyze'], 
              i['Nken_Analyze'],
              i['Nwa_Analyze']]for i in wowDatas if 'NbusStation_Analyze' in i and 'avgDailyCustomer' in i and 'costPower_Analyze' in i
            ])
#             and i['Called']==store])
#               and i['typeP']==store])


# ,
#               i['Nstar_Analyze'], 
#               i['Nmc_Analyze'], 
#               i['Nken_Analyze'],
#               i['Nwa_Analyze']
#,  i['Nstar_Analyze'],  i['Nmc_Analyze'],  i['Nken_Analyze'],  i['costPower_Analyze'],    i['Nhuman_Analyze'],    i['NbusStation_Analyze']  i['NconStore_Analyze']

Y= np.array([i['type'] for i in wowDatas if 'NbusStation_Analyze' in i and 'avgDailyCustomer' in i and 'costPower_Analyze' in i
            ])
#             and i['Called']==store])
#              and i['typeP']==store])
#avgDailyCustomer 'type'


In [ ]:
##### from sklearn.cross_validation import train_test_split
#將資料分成訓練、測試集 其中測試集佔三成
data_train, data_test, labels_train, labels_test =train_test_split(X,Y,test_size=0.3,random_state=1)

from sklearn.preprocessing import StandardScaler
#對數據做正規化的動作(減去平均值、縮放成一單位)，可以改進效能!!!
scarler=StandardScaler()
scarler.fit(X)
data_train_std=scarler.transform(data_train)
data_test_std=scarler.transform(data_test)

In [ ]:
# mlp=MLPClassifier(random_state=1)
mlp=MLPClassifier(random_state=1,hidden_layer_sizes=(200,500,300),activation="relu",max_iter=500)
# mlp=MLPClassifier(random_state=1,hidden_layer_sizes=(1000,1000,1000,1000,1000),activation="relu",max_iter=800)

# mlp=MLPRegressor(random_state=1,hidden_layer_sizes=(200,500,300),activation="relu",max_iter=500)

mlp.fit(data_train_std,labels_train)

In [ ]:
#本身
pred=mlp.predict(data_train_std)
print("自己")
print("Misclassified samples: {}".format((labels_train != pred).sum()))
print("Accuracy: {}".format(accuracy_score(labels_train,pred)))

print("="*50)
#預測
print("測試")
pred=mlp.predict(data_test_std)
print("Misclassified samples: {}".format((labels_test != pred).sum()))
print("Accuracy: {}".format(accuracy_score(labels_test,pred)))

In [ ]:
pred

In [ ]:
newData=[[int(j)for j in i.split("\t")] for i in """25\t17957\t75\t49""".split('\n')]
# 新店資料

newData_std=scarler.transform(newData)


# print('消費力	人口數	公車站數	四大超商數	星巴克數	麥當勞數	肯德基數	瓦城數')

to_be_predicted = np.array(newData_std)
predicted_sales=mlp.predict(to_be_predicted)
predicted_sales

# try keras neural network

In [ ]:
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers.core import Dense,Activation
from keras.utils import np_utils
import pymongo


In [ ]:
import pymongo
client=pymongo.MongoClient('192.168.1.113',27017,username="j122085",password="850605")

db=client.rawData
print(db.collection_names())
collection=db.wowprimediendata
wowDatas=list(collection.find({}))

Calls=set(i["Called"] for i in wowDatas)
for Call in Calls:
    mean=np.mean([i["avgDailyCustomer"] for i in wowDatas if i["Called"]==Call and 'avgDailyCustomer' in i])
    if not np.isnan(mean):
        for j in wowDatas:
            if j["Called"]==Call and 'avgDailyCustomer' in j:
                if j["avgDailyCustomer"]>mean*1.1:
                    j['type']=0#"good"
                elif j["avgDailyCustomer"]<mean*0.9:
                    j['type']=2#"bad"
                else:
                    j['type']=1#"normal"
            if 'avgDailyCustomer' in j:
                aC=j["avgDailyNet"]/j["avgDailyCustomer"]
                if aC<290:
                    j['typeP']="低"
                elif aC<450:
                    j['typeP']="中"
                elif aC<800:
                    j['typeP']="中高"
                else:
                    j['typeP']="高"
                

store=input("輸入店家")
import numpy as np
from sklearn.linear_model import LinearRegression
X= np.array([[i['costPower_Analyze'],
              i['Nhuman_Analyze'],
              i['NbusStation_Analyze'],
              i['NconStore_Analyze'],
              i['Nstar_Analyze'], 
              i['Nmc_Analyze'], 
              i['Nken_Analyze'],
              i['Nwa_Analyze']]for i in wowDatas if 'NbusStation_Analyze' in i and 'avgDailyCustomer' in i and 'costPower_Analyze' in i
            ])
#             and i['Called']==store])
#               and i['typeP']==store])


# ,
#               i['Nstar_Analyze'], 
#               i['Nmc_Analyze'], 
#               i['Nken_Analyze'],
#               i['Nwa_Analyze']
#,  i['Nstar_Analyze'],  i['Nmc_Analyze'],  i['Nken_Analyze'],  i['costPower_Analyze'],    i['Nhuman_Analyze'],    i['NbusStation_Analyze']  i['NconStore_Analyze']

Y= np.array([i['type'] for i in wowDatas if 'NbusStation_Analyze' in i and 'avgDailyCustomer' in i and 'costPower_Analyze' in i
            ])
#             and i['Called']==store])
#              and i['typeP']==store])
#avgDailyCustomer 'type'


In [ ]:
classes=3
Y_train=np_utils.to_categorical(Y,classes)

In [ ]:
input_size=8
batch_size=5
hidden_neurons=6
epochs=50

In [ ]:
model=Sequential()  #定義model
model.add(Dense(20,input_dim=input_size)) 
model.add(Activation('relu')) #啟動函數
model.add(Dense(15)) 
model.add(Activation('relu')) #啟動函數
model.add(Dense(10)) 
model.add(Activation('sigmoid')) #啟動函數
model.add(Dense(5)) 
model.add(Activation('sigmoid')) #啟動函數
model.add(Dense(1))  #加入層(緊密層) 產出個數10.輸入個數100 次元
model.add(Activation("linear"))
model.compile(loss="MSE",metrics=['accuracy'],optimizer='sgd')
model.fit(X,Y_train,batch_size=batch_size,nb_epoch=epochs,verbose=1)


In [ ]:
score=model.evaluate(X,Y_train,verbose=1)
print('Test accuracy:',score[1])


In [ ]:
newData=[[int(j)for j in i.split("\t")] for i in """63	282141	204	187	16	14	2	1""".split('\n')]
model.predict(numpy.array(newData).astype('float32'))

# scipy linear

In [ ]:
import pymongo
client=pymongo.MongoClient('192.168.1.113',27017,username="j122085",password="850605")

db=client.rawData
print(db.collection_names())
collection=db.wowprimediendata
wowDatas=list(collection.find({}))
                

store=input("輸入店家")
import numpy as np
from sklearn.linear_model import LinearRegression
X= np.array([[i['costPower_Analyze'],
              i['Nhuman_Analyze'],
              i['NbusStation_Analyze'],
              i['NconStore_Analyze'],
              i['Nstar_Analyze'], 
              i['Nmc_Analyze'], 
              i['Nken_Analyze'],
              i['Nwa_Analyze']]for i in wowDatas if 'NbusStation_Analyze' in i and 'avgDailyCustomer' in i and 'costPower_Analyze' in i
#             ])
            and i['Called']==store])
#               and i['typeP']==store])


# ,
#               i['Nstar_Analyze'], 
#               i['Nmc_Analyze'], 
#               i['Nken_Analyze'],
#               i['Nwa_Analyze']
#,  i['Nstar_Analyze'],  i['Nmc_Analyze'],  i['Nken_Analyze'],  i['costPower_Analyze'],    i['Nhuman_Analyze'],    i['NbusStation_Analyze']  i['NconStore_Analyze']

Y= np.array([i['avgDailyCustomer'] for i in wowDatas if 'NbusStation_Analyze' in i and 'avgDailyCustomer' in i and 'costPower_Analyze' in i
#             ])
            and i['Called']==store])
#              and i['typeP']==store])
#avgDailyCustomer 'type'


In [ ]:
dienCalled='原燒	藝奇	王品	CooK BEEF!	麻佬大	hot 7	陶板屋	品田牧場	舒果	聚	夏慕尼	ita義塔	莆田	石二鍋	ＴＡＳＴｙ	乍牛	沐越	青花驕'.split("\t")
dienCalled

In [ ]:
features=['costPower_Analyze','Nhuman_Analyze','NbusStation_Analyze','NconStore_Analyze','Nstar_Analyze', 'Nmc_Analyze', 'Nken_Analyze','Nwa_Analyze']
features


featuremap={'costPower_Analyze':"消費力",'Nhuman_Analyze':"人口",'NbusStation_Analyze':"公車站",'NconStore_Analyze':"便利商店數",'Nstar_Analyze':"星巴克", 'Nmc_Analyze':"麥當勞", 'Nken_Analyze':"肯德基",'Nwa_Analyze':"瓦城"}

In [ ]:
#coding:utf-8
import numpy as np
from scipy.optimize import curve_fit
import matplotlib.pyplot as plt
plt.rcParams['font.sans-serif']=['SimHei'] #用来正常显示中文标签
plt.rcParams['axes.unicode_minus']=False #用来正常显示负号
#有中文出现的情况，需要u'内容'


for store in dienCalled:
    for feature in features:
        X= np.array([i[feature] for i in wowDatas if 'NbusStation_Analyze' in i and 'avgDailyCustomer' in i and 'costPower_Analyze' in i
                and i['Called']==store])
        Y= np.array([i['avgDailyCustomer'] for i in wowDatas if 'NbusStation_Analyze' in i and 'avgDailyCustomer' in i and 'costPower_Analyze' in i
                and i['Called']==store])


        xdata=X
        ydata=Y
        def func(x,a,b):
            return a+b*x

        try:
            popt, pcov=curve_fit(func,xdata,ydata)
        except:
            continue
        plt.xlabel(featuremap[feature])
        plt.ylabel('來客數')
        plt.title(store)
        plt.plot(xdata,ydata,'ko',label='Orifinal Noised Data')
        plt.plot(xdata,func(xdata,*popt),'r',label='Fitted Curve')
        plt.show()